In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import sys
import csv
import os
from sklearn import metrics
from dateutil.parser import parse
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import lightgbm as lgb
%matplotlib inline

In [2]:
import numpy as np, pandas as pd, os
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
import random
import string

In [3]:
data_version_input = "01-10-2020"

In [4]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version_input}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version_input}/test.csv")

/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_transform = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis = 0)
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G"],
    
    "cat_str" : ["Field_3","Field_4", "Field_12","Field_36", "Field_38", "Field_54", "Field_62","Field_65", \
                 "Field_66", "info_social_sex", "brief", "gioiTinh"], 
    
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40","Field_43", "Field_44", \
              "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate", "ngaySinh"],
    
    "text" : ["Field_18","currentLocationCity", "homeTownCity", "Field_48", "Field_49",\
             "currentLocationCountry", 'homeTownState','data.basic_info.locale', "currentLocationName", \
              'homeTownName', "homeTownCountry", "maCv", "currentLocationState", "Field_61", "Field_56",],
    
    "drop" : ["Field_45", "Field_34","Field_68", "diaChi", "Field_46", "Field_55"]
}

df_transform = df_transform.drop(data_type["drop"], axis = 1)
df_transform = df_transform.drop(data_type["text"], axis = 1)
df_transform = df_transform.drop(data_type["date"], axis = 1)
for feature in  data_type["cat_num"]:
    df_transform[feature] = df_transform[feature].fillna("Nan")
    df_transform = pd.get_dummies(df_transform, columns=[feature])
for feature in  data_type["cat_str"]:
    df_transform[feature] = df_transform[feature].fillna("Nan")
    df_transform = pd.get_dummies(df_transform, columns=[feature])
"""    
for feature in data_type["date"]:
    df_transform[feature] = df_transform[feature].map(lambda x: float(parse(str(x)).timestamp()) if type(x) is str else x)
    df_transform[feature] = df_transform[feature].fillna(df_transform[feature].mean())
"""

df_transform = df_transform.fillna(df_transform.mean())

## Dimensionality reduction
pca = PCA(n_components = 100)
sel = pca.fit(df_transform)
df_transform = pd.DataFrame(sel.transform(df_transform))

train_dev = df_transform.iloc[:len(df_train),:]
test = df_transform.iloc[len(df_train):,:]

scaler = MinMaxScaler()
scaler.fit(train_dev)
X_transform = scaler.transform(train_dev)
X_transform = pd.DataFrame(data = X_transform)

test_transform = scaler.transform(test)

test = pd.DataFrame(data = test_transform)
train_dev = pd.concat([df_train.id, df_train.label, X_transform], axis = 1)
test = pd.concat([df_test.id, test], axis = 1)
train_dev.reset_index(drop=True,inplace=True)

In [ ]:
result_str = ''.join(random.choice(string.ascii_letters) for i in range(5))
data_version = f"06-10-2020-{result_str.lower()}"
print(data_version)

In [ ]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
train.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)